In [9]:
import json
from rouge import Rouge
import os
import pandas as pd
import numpy as np
import copy
import jieba
import sys
import re
import logging
import gensim
from gensim.models.word2vec import LineSentence
from sklearn import model_selection
from config import config_base
import vocab
from preprocess_data import *
config = config_base.config

In [10]:
df = organize_data(config.train_data)
df = df.sample(5000)
df = deal_data(df)

In [12]:
df = shorten_content_all(df, config.max_len)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.442 seconds.
Prefix dict has been built succesfully.


shorten content, accuracy: 0.9666
max length: 300
min length: 9
mean length:181
median length:177


In [23]:
build_answer_range(df)

answer generation accuracy(all): 0.9226
wrong split: 0.0379
answer duplicate: 0.0395


,article_id,article_type,article_title,article_content,question_id,article_question,article_answer,question_type,title,content,question,answer,merge,is_in,len,answer_start,answer_end
20974,2127,新闻,美军向印太地区增派2艘驱逐舰 协助黄蜂号执行任务,【环球网综合报道】俄罗斯卫星网2月7日援引美国海军新闻稿报道称，该国派出“杜威”号和“斯...,a525d125-3969-4c53-b7c8-25e0e354c7ae,【谁综合报道】俄罗斯卫星网2月7日援引美国海军新闻稿报道称，该国派出“杜威”号和“斯特雷特”...,环球网,篇章型问题,美军向印太地区增派2艘驱逐舰 协助黄蜂号执行任务,【环球网综合报道】俄罗斯卫星网2月7日援引美国海军新闻稿报道称，该国派出“杜威”号和“斯特雷...,【谁综合报道】俄罗斯卫星网2月7日援引美国海军新闻稿报道称，该国派出“杜威”号和“斯特雷特”...,环球网,美军向印太地区增派2艘驱逐舰 协助黄蜂号执行任务。【环球网综合报道】俄罗斯卫星网2月7日援引...,True,122,16.0,16.0
5336,6873,新闻,英国要求从利比亚引渡曼彻斯特爆炸案恐怖分子,中新网11月2日电据外媒报道，英国曼彻斯特警方表示，英国当局向利比亚请求考虑引渡萨勒曼·阿贝...,db16a35f-0e6f-4ebf-8d72-0d4b30b29f71,英国要求从哪里引渡曼彻斯特爆炸案恐怖分子,利比亚,事实型问题,英国要求从利比亚引渡曼彻斯特爆炸案恐怖分子,中新网11月2日电据外媒报道，英国曼彻斯特警方表示，英国当局向利比亚请求考虑引渡萨勒曼·阿贝...,英国要求从哪里引渡曼彻斯特爆炸案恐怖分子,利比亚,英国要求从利比亚引渡曼彻斯特爆炸案恐怖分子。中新网11月2日电据外媒报道，英国曼彻斯特警方表...,True,127,3.0,3.0
50662,46424,防务快讯,外媒：三次试射失败为美反导系统敲响警钟,核心提示： 陆基中段防御系统在拦截测试中成功率并不高。在16次测试中，该系统成功摧毁目标导弹...,b93724d2-b6e6-4321-ad57-06511ee96fa4,几次试射失败为美反导系统敲响警钟,三次,数值型问题,外媒：三次试射失败为美反导系统敲响警钟,核心提示： 陆基中段防御系统在拦截测试中成功率并不高。在16次测试中，该系统成功摧毁目标导弹...,几次试射失败为美反导系统敲响警钟,三次,外媒：三次试射失败为美反导系统敲响警钟。核心提示： 陆基中段防御系统在拦截测试中成功率并不高...,True,256,2.0,2.0
94031,3111,新闻,韩朝商朝鲜艺术团参奥技术事宜 下午将继续讨论,中新网1月15日电 据韩联社15日报道，韩国统一部消息称，韩朝双方于今日10点10分和1...,4c4d12af-1c3a-4077-8280-a0d2b95843bf,韩国统一部消息称，韩朝双方于什么时候在边境板门店依次进行全体会议和代表会议，就演出时间、地点...,今日10点10分和12点,数值型问题,韩朝商朝鲜艺术团参奥技术事宜 下午将继续讨论,中新网1月15日电 据韩联社15日报道，韩国统一部消息称，韩朝双方于今日10点10分和12点...,韩国统一部消息称，韩朝双方于什么时候在边境板门店依次进行全体会议和代表会议，就演出时间、地点...,今日10点10分和12点,韩朝商朝鲜艺术团参奥技术事宜 下午将继续讨论。中新网1月15日电 据韩联社15日报道，韩国统...,True,160,34.0,41.0
4290,29877,新闻,英国军方将军队武装无人机数量翻倍,参考消息网12月6日报道 外媒称，英国国防大臣迈克尔法伦3日宣布，根据和美国无人机制造商通用...,3161ab9f-3c4e-4c7c-8615-4eb7b605efcc,据英国《卫报》网站12月3日报道称什么,美国广泛使用的“捕食者”和“死神”无人机的制造商通用原子能公司将向英国皇家空军提供10架无人...,观点型问题,英国军方将军队武装无人机数量翻倍,参考消息网12月6日报道 外媒称，英国国防大臣迈克尔法伦3日宣布，根据和美国无人机制造商通用...,据英国《卫报》网站12月3日报道称什么,美国广泛使用的“捕食者”和“死神”无人机的制造商通用原子能公司将向英国皇家空军提供10架无人...,英国军方将军队武装无人机数量翻倍。参考消息网12月6日报道 外媒称，英国国防大臣迈克尔法伦3...,True,171,69.0,103.0
85191,10272,新闻,辽宁舰首次进行实兵实弹演习 歼15挂实弹打击目标(组图),飞行员向起甲板保障人员示意可以起飞。张凯-摄影 国际在线消息（记者 杨琼）：记者15日从...,7bea08a7-bb40-4592-877b-fdb9085f7b37,中国海军组织了什么演习,实施了航母编队实际使用武器演习,事实型问题,辽宁舰首次进行实兵实弹演习 歼15挂实弹打击目标(组图),飞行员向起甲板保障人员示意可以起飞。张凯-摄影 国际在线消息（记者 杨琼）：记者15日从中国...,中国海军组织了什么演习,实施了航母编队实际使用武器演习,辽宁舰首次进行实兵实弹演习 歼15挂实弹打击目标(组图)。飞行员向起甲板保障人员示意可以起飞...,True,190,54.0,61.0
899,12740,新闻,国防部回应“日美将在南海举行联合演习”：高度关注,【环球网综合报道】中国国防部12月31日召开例行记者会，国防部新闻事务局局长杨宇军回答记...,5a8c8708-1b4e-421b-9420-93726f96e97d,有关什么之间的军事安全合作，不应破坏地区的和平与稳定。,国家,篇章型问题,国防部回应“日美将在南海举行联合演习”：高度关注,【环球网综合报道】中国国防部12月31日召开例行记者会，国防部新闻事务局局长杨宇军回答记者提...,有关什么之间的军事安全合作，不应破坏地区的和平与稳定。,国家,国防部回应“日美将在南海举行联合演习”：高度关注。【环球网综合报道】中国国防部12月31日召...,True,115,100.0,100.0
54834,31562,新闻,美国海军上演惊天腐败大戏：将军沦陷 小兵也猖狂,图为伦纳德弗朗西斯(中)。迈克尔米西维兹 新年伊始，曾经轰动一时的美国海军腐败案持续发酵。...,a1acd20d-7b93-49e5-9165-4f9ef0ed07bc,海军中士丹尼尔拉尤格因什么被判27个月的监禁,收受贿赂、出卖海军机密情报、危及国家安全,事实型问题,美国海军上演惊天腐败大戏：将军沦陷 小兵也猖狂,图为伦纳德弗朗西斯(中)。迈克尔米西维兹新年伊始，曾经轰动一时的美国海军腐败案持续发酵。1月...,海军中士丹尼尔拉尤格因什么被判27个月的监禁,收受贿赂、出卖海军机密情报、危及国家安全,美国海军上演惊天腐败大戏：将军沦陷 小兵也猖狂。图为伦纳德弗朗西斯(中)。迈克尔米西维兹新年...,True,132,62.0,71.0
51609,9543,新闻,精锐尽出！美韩海军派逾60舰演习 包括核动力航母(图),美军核动力航空母舰“卡尔文森号” 【环球网综合报道】朝鲜半岛局势波谲云诡，美国和韩国正举行...,fe831555-549c-4f85-8fd6-0cbe85155d75,举行联合演习的目的,应对朝鲜挑衅,观点型问题,精锐尽出！美韩海军派逾60舰演习 包括核动力航母(图),美军核动力航空母舰“卡尔文森号” 【环球网综合报道】朝鲜半岛局势波谲云诡，美国和韩国正举行年...,举行联合演习的目的,应对朝鲜挑衅,精锐尽出！美韩海军派逾60舰演习 包括核动力航母(图)。美军核动力航空母舰“卡尔文森号” 【...,True,293,97.0,99.0
16309,8631,新闻,韩“萨德”联合专案组开会 协调国防外交环境三部立场,【环球网报道 记者 李小飞】韩联社6月8日报道称，由韩国国防部、外交部和环境部副部级高官...,91c2dcfb-da3d-4cea-9a39-6fcb43756ca4,韩国总统府青瓦台高官日前表示，本应在部署“萨德”前进行战略环境影响评价，但国防部未与谁磋商便...,环境部,事实型问题,韩“萨德”联合专案组开会 协调国防外交环境三部立场,【环球网报道 记者 李小飞】韩联社6月8日报道称，由韩国国防部、外交部和环境部副部级高官及部...,韩国总统府青瓦台高官日前表示，本应在部署“萨德”前进行战略环境影响评价，但国防部未与谁磋商便...,环境部,韩“萨德”联合专案组开会 协调国防外交环境三部立场。【环球网报道 记者 李小飞】韩联社6月8...,True,159,121.0,121.0


In [22]:
def build_answer_range(df):
    sys.setrecursionlimit(1000000)
    rouge = Rouge(metrics=['rouge-l'])

    def match(merge, answer, question):
        merge_list = jieba.lcut(merge)
        merge_len = len(merge_list)
        answer_list = jieba.lcut(answer)
        answer_len = len(answer_list)
        question_str = ' '.join(jieba.lcut(question))
        start = []
        end = []
        for i in range(0, merge_len-answer_len+1):
            if merge_list[i: i+answer_len] == answer_list:
                start.append(i)
                end.append(i+answer_len-1)
        if len(start) == 0:
            return -1, -1
        elif len(start) == 1:
            return start[0], end[0]
        else:
            scores = []
            # 前后扩展5个词
            for s, e in zip(start, end):
                s = max(s-5, 0)
                answer_can = ' '.join(merge_list[s: e+5])
                score = rouge.get_scores(answer_can, question_str, avg=True)['rouge-l']['r']
                scores.append(score)
            max_score = max(scores)
            start_tmp = []
            end_tmp = []
            for i in range(len(scores)):
                if scores[i] == max_score:
                    start_tmp.append(start[i])
                    end_tmp.append(end[i])
            start = start_tmp
            end = end_tmp
            if len(start) == 1:
                return start[0], end[0]
            else:
                return -2, -2

    merges = df[df['is_in']]['merge'].values
    answers = df[df['is_in']]['answer'].values
    questions = df[df['is_in']]['question'].values
    answer_range = [match(m, a, q) for m, a, q in zip(merges, answers, questions)]

    start, end = list(zip(*answer_range))
    df.loc[df['is_in'], 'answer_start'] = start
    df.loc[df['is_in'], 'answer_end'] = end

    merge_len = len(merges)
    right_all_len = (df['answer_end'] >= 0).sum()
    wrong_split_len = (df['answer_end'] == -1).sum()
    wrong_dup_len = (df['answer_end'] == -2).sum()
    print('answer generation accuracy(all): %.4f' % (right_all_len/merge_len))
    print('wrong split: %.4f' % (wrong_split_len/merge_len))
    print('answer duplicate: %.4f' % (wrong_dup_len/merge_len))

    return df